In [1]:
import os
import re
import csv
import pandas as pd
import json
import numpy as np
import heapq
from matplotlib import colors
import matplotlib.pyplot as plt
import torch
import networkx as nx
from networkx.readwrite import json_graph
from graph_to_prompt import graph_to_prompt
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

In [2]:
def get_nums_list(s):
    nums_str = re.findall(r'\d+', s)  
    nums = list(map(int, nums_str))
    return  nums

In [3]:
class Request_llm:
    def __init__(self, model_name):
        if model_name == 'llama':
            self.max_new_tokens = 200
            self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
            self.model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", device_map="auto",load_in_8bit=True)
            #self.model.save_model("models/meta-llama/Llama-3.2-3B-Instruct")
        

        self.model.config.pad_token_id = self.model.config.eos_token_id

    def get_waypoints(self, j, start, goal, n_points):
        prompt = graph_to_prompt(j, start, goal, n_points)
        inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    return_attention_mask=True
                )
        
        input_ids = inputs["input_ids"]
        inputs["input_ids"] = inputs["input_ids"].to(self.model.device)
        inputs["attention_mask"] = inputs["attention_mask"].to(self.model.device)

        outputs = self.model.generate(
                **inputs,
                max_new_tokens = self.max_new_tokens
            )
        generated_token = outputs[:, input_ids.shape[-1] :]
        result = self.tokenizer.batch_decode(generated_token, skip_special_tokens=True)[0]
        waypoints = get_nums_list(result)
        return waypoints


In [4]:
def astar(G, start, goal, heuristic, obstacles = None):
    count = 0
    storage = 0
    checking_edges = set()

    if obstacles is None:
        obstacles = set()
    else:
        obstacles = set(obstacles)

    # g-score 와 f-score 초기화
    g_score = {n: float('inf') for n in G.nodes}
    g_score[start] = 0

    f_score = {n: float('inf') for n in G.nodes}
    f_score[start] = heuristic[start]

    # 우선순위 큐: (f_score, g_score, node)
    open_set = []
    heapq.heappush(open_set, (f_score[start], 0, start))
    visited = set()
    came_from = {}

    num_iter = 0

    while open_set:
        num_iter += 1
        storage += len(open_set) + len(visited) + len(came_from)
        f_cur, g_cur, current = heapq.heappop(open_set)

        # 목표 도달
        if current == goal:
            # 경로 재구성
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            return path[::-1], g_cur, count, storage, list(checking_edges)
        visited.add(current)
        # 이웃 탐색
        for nbr in G.neighbors(current):
            count = count + 1
            if nbr in visited or nbr in obstacles:
                continue
            
            tentative_g = g_score[current] + G[current][nbr]['weight']
            checking_edges.add((current, nbr))
            if tentative_g < g_score[nbr]:
                came_from[nbr] = current
                g_score[nbr] = tentative_g
                f_score[nbr] = tentative_g + heuristic[nbr]
                heapq.heappush(open_set, (f_score[nbr], tentative_g, nbr))

    # 경로가 없을 때
    return None, None, count, storage, list(checking_edges)

In [5]:
def llm_astar(G, start, goal, heuristic, llm, obstacles=None):
    nodes = list(G.nodes)
    llm = list(set(llm))
    T = []
    for ts in llm:
        if ts in nodes:
            T.append(ts)
    if obstacles is None:
        obstacles = set()
    else:
        obstacles = set(obstacles)
    print(llm)
    # T.reverse()
    if not T or T[0] != start:
        T.insert(0, start)
    if T[-1] != goal:
        T.append(goal)
    T = [t for t in T if t not in obstacles]
    t_idx = 1 if len(T) > 1 else 0
    t = T[t_idx]
    dist_to_t = nx.single_source_dijkstra_path_length(G, t, weight='weight')
    count = 0
    storage = 0
    checking_edges = set()
    came_from = {}
    visited = set()

    g_score = {n: float('inf') for n in G.nodes}
    g_score[start] = 0

    f_score = {n: float('inf') for n in G.nodes}
    f_score[start] = g_score[start] + heuristic[start] + dist_to_t.get(start, float('inf'))

    open_set = []
    heapq.heappush(open_set, (f_score[start], g_score[start], start))

    while open_set:
        storage += len(open_set) + len(visited) + len(came_from)
        f_cur, g_cur, current = heapq.heappop(open_set)
        if current == goal:
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            return path[::-1], g_cur, count, storage, list(checking_edges), T
        visited.add(current)
        if current == t and t_idx < len(T) - 1:
            t_idx += 1
            t = T[t_idx]
            dist_to_t = nx.single_source_dijkstra_path_length(G, t, weight='weight')

            new_open = []
            for _, g_old, node in open_set:
                new_f = g_score[node] + heuristic[node] + dist_to_t.get(node, float('inf'))
                new_open.append((new_f, g_old, node))
            open_set = new_open
            heapq.heapify(open_set)


        for nbr in G.successors(current):
            if nbr in visited or nbr in obstacles:
                continue
            count += 1
            w = G[current][nbr].get('weight', 1)
            tentative_g = g_score[current] + w
            checking_edges.add((current, nbr))

            if tentative_g < g_score[nbr]:
                came_from[nbr] = current
                g_score[nbr] = tentative_g
                f = tentative_g + heuristic[nbr] + dist_to_t.get(nbr, float('inf'))
                f_score[nbr] = f
                heapq.heappush(open_set, (f, tentative_g, nbr))


    return None, None, count, storage, list(checking_edges), T


In [6]:
with open("graphs/sejong_bus.json", "r") as json_file:
    j = json.load(json_file)

G = json_graph.node_link_graph(j)
nodes = G.nodes
edges = G.edges
stations = list(G.nodes())

/opt/conda/envs/LLM/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [12]:
# table(random) A*
experiments_samples_A_star = []

while True:
    start, goal = np.random.choice(nodes, 2)
    start, goal = start.item(), goal.item()

    heuristic_table = {station: (((G.nodes[goal]['x'] - G.nodes[station]['x']) ** 2 + (G.nodes[goal]['y'] - G.nodes[station]['y']) ** 2) ** 0.5) for station in nodes}

    path, cost, count, storage, checking_edges = astar(G, start, goal, heuristic_table)
    if path is not None:
        experiments_samples_A_star.append({'point' : (start, goal), 'path':path, 'cost':cost, 'storage':storage, 'count':count, 'checking_edges' : len(checking_edges)})
    if len(experiments_samples_A_star)>150:
        break

In [8]:
llm_model = Request_llm('llama')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# table(random) LLM-A*
num_request = 7

experiments_samples_LLM_A_star = []

for i, sample in (enumerate(tqdm(experiments_samples_A_star[:10]))):
    start, goal = sample['point']

    heuristic_table = {station: (((G.nodes[goal]['x'] - G.nodes[station]['x']) ** 2 + (G.nodes[goal]['y'] - G.nodes[station]['y']) ** 2) ** 0.5) for station in nodes}

    waypoints = llm_model.get_waypoints(j, start, goal, num_request)
    
    path, cost, count, storage, checking_edges, waypoints_using_llm = llm_astar(G, start, goal, heuristic_table, waypoints)
    if path is not None:
        experiments_samples_LLM_A_star.append(
            {'point' : (start, goal), 
             'path':path, 
             'cost':cost, 
             'count':count, 
             'storage':storage,
             'checking_edges' : len(checking_edges),
             'waypoints_of_llm' : waypoints,
             'waypoints_using_llm' : waypoints_using_llm,
             'success' : (len(waypoints) <= num_request + 1)
             })
        cost_ratio = 1 - cost/sample['cost']
        count_ratio = 1 - count/sample['count']
        storage_ratio = 1 - storage/sample['storage']
        checking_edges_ratio = 1 - len(checking_edges)/sample['checking_edges']
        print("Sample ", i)
        print("Count ratio       :", count_ratio)
        print("Storage ratio    ",  storage_ratio)
        print("Cost ratio        :", cost_ratio)
        print("# of Checking edges ratio:", checking_edges_ratio)

  0%|          | 0/10 [00:00<?, ?it/s]

/opt/conda/envs/LLM/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[6806309, 6806344, 6806345, 6806346, 6806347, 6806330, 6806331]
Sample  0
Count ratio       : 0.9147727272727273
Storage ratio     0.9830363429140458
Cost ratio        : 0.0
# of Checking edges ratio: 0.8265895953757225


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[6801908, 6801910, 6801912, 6801914, 6801915, 6801916, 6801917]
Sample  1
Count ratio       : 0.9430479183032208
Storage ratio     0.9898253670100564
Cost ratio        : 0.0
# of Checking edges ratio: 0.8822095857026807


...

[6805511, 6805512, 6805513, 6805514, 6805515, 6805516, 6805517]
Sample  6
Count ratio       : 0.5052882975093824
Storage ratio     -0.017327358726715092
Cost ratio        : 0.0
# of Checking edges ratio: 0.0


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[6800001, 6800002, 6800003, 6800004, 6800005, 6800006, 6800632]
Sample  7
Count ratio       : 0.6814044213263979
Storage ratio     0.6507642835117395
Cost ratio        : 0.0
# of Checking edges ratio: 0.3893333333333333


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[6800833, 6800836, 6800839, 6800842, 6800816, 6800820, 6800823, 6800826, 6800830]
Sample  8
Count ratio       : 0.5035552682611506
Storage ratio     -0.014111983974526732
Cost ratio        : 0.0
# of Checking edges ratio: -0.002670226969292422
[6801172, 6801173, 6801174, 6801175, 6801176, 6801177, 6801178]
Sample  9
Count ratio       : 0.48500254194204373
Storage ratio     -0.04715323563149876
Cost ratio        : 0.0
# of Checking edges ratio: -0.01747173689619741


In [13]:
data = experiments_samples_LLM_A_star

fields = ['point', 'path', 'cost', 'count', 'checking_edges', 'waypoints_of_llm', 'waypoints_using_llm', 'success']

with open('Experiments_result/LLM_A_star_fewshot.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

ValueError: dict contains fields not in fieldnames: 'storage'

In [ ]:
data = pd.read_csv('Experiments_result/A_star.csv')
data.loc[0].

point                                                 (6800376, 6800463)
path                   [6800376, 6800375, 6800374, 6800373, 6800372, ...
cost                                                                  34
count                                                               1159
checking_edges                                                      1108
waypoints_of_llm                                                     NaN
waypoints_using_llm                                                  NaN
Name: 0, dtype: object

In [ ]:
data = []

In [44]:
import ast
ast.literal_eval(data[0]['point'])

(6800376, 6800463)

In [16]:
experiments_samples_A_star[0]

{'point': (6806188, 6805679),
 'path': [6806188,
  6800392,
  6800391,
  6805646,
  6800389,
  6800388,
  6800387,
  6800386,
  6800385,
  6800384,
  6800624,
  6800359,
  6802337,
  6800358,
  6800357,
  6800356,
  6800355,
  6800354,
  6800353,
  6800352,
  6800351,
  6800337,
  6800350,
  6800336,
  6800335,
  6800348,
  6800347,
  6800346,
  6800345,
  6800344,
  6800343,
  6800342,
  6800340,
  6800339,
  6800338,
  6800160,
  6801208,
  6801849,
  6801139,
  6801799,
  6800165,
  6800006,
  6805261,
  6801864,
  6801866,
  6801817,
  6800016,
  6801874,
  6801165,
  6800626,
  6801145,
  6801146,
  6806307,
  6806308,
  6806309,
  6801843,
  6801163,
  6802358,
  6805679],
 'cost': 83,
 'storage': 1007604,
 'count': 2445,
 'checking_edges': 1188}

In [14]:
with open('Experiments_result/A_star.json', 'w') as jsonfile:
	json.dump(experiments_samples_A_star, jsonfile, indent=4)

In [11]:
with open('Experiments_result/A_star.json', 'r') as jsonfile:
    data = json.load(jsonfile)

print(data)

[{'point': [6806309, 6800341], 'path': [6806309, 6806308, 6806307, 6801146, 6801145, 6800626, 6801165, 6801874, 6800016, 6801817, 6801866, 6801864, 6805261, 6800006, 6800165, 6801799, 6801139, 6801849, 6801208, 6800383, 6800154, 6800341], 'cost': 38, 'storage': 514158, 'count': 1760, 'checking_edges': 865}, {'point': [6801908, 6800482], 'path': [6801908, 6801909, 6806332, 6806333, 6802350, 6802399, 6801860, 6801862, 6805260, 6800040, 6801257, 6802339, 6800291, 6800290, 6800289, 6800288, 6800287, 6800286, 6800907, 6800260, 6802330, 6800261, 6802331, 6800247, 6800246, 6805639, 6800245, 6801804, 6800244, 6805633, 6805631, 6801277, 6800488, 6800487, 6800579, 6800485, 6801841, 6800484, 6800483, 6801275, 6800482], 'cost': 49, 'storage': 1125446, 'count': 2546, 'checking_edges': 1231}, {'point': [6800689, 6800614], 'path': [6800689, 6801227, 6800687, 6806327, 6800686, 6800685, 6800308, 6800309, 6801244, 6800322, 6800114, 6806359, 6806360, 6800949, 6800950, 6800961, 6800962, 6806273, 6800755, 